# multi-threading com julia ༜

### ⋆ controle do número de threads
1. argumento de comando: `-t` ou `--threads`
2. variável de ambiente: `JULIA_NUM_THREADS`

obs: pode ser testado com `Threads.nthreads()`

### ⋆ funções e macros
  *Threads.threadid()*
    
  Returna id da thread em execução (thread mestre tem id=1)

  *Thread.@spawn t*

  Aloca a task t para uma thread disponível
  
  Pode ser alocada para mais de uma thread em caso de loop

  *fetch*
  
  Bloqueia a execução do programa até que a tarefa termine

In [ ]:
x = Threads.@spawn begin
    print("thread ", Threads.threadid(), " somou: " ) #apenas uma thread irá realizar a soma
    sum(1:1_000_000)
end

print(fetch(x))

In [ ]:
tasks = []
for i in 1:4
    push!(tasks, Threads.@spawn begin # cada thread irá realizar a soma de 1000 elementos por vez
            println(Threads.threadid())
            sum((i-1)*1_000+1:i*1_000)
        end)
end

total = sum(fetch.(tasks)) # o ponto é para aplicar a função fetch a cada elemento do vetor tasks
print(total)

  *Thread.@threads*

  Usada para paralelizar loops, dividindo automaticamente as iterações entre as threads disponíveis

  Não precisa de `fetch`, pois ela garante que todas as iterações serão executadas

In [ ]:
Threads.@threads for i in 1:4
    println("Thread $(Threads.threadid()) executando iteração $i")
end

In [ ]:
sum = 0
Threads.@threads for i in 1:10
    global sum += i
end

println(sum)

### ⋆ condições de corrida

Ocorrem em programas paralelos quando múltiplas threads ou processos tentam acessar ou modificar uma mesma variável ou recurso ao mesmo tempo. O resultado final pode depender da **ordem de execução das threads**, que não é garantida. Operações intermediárias podem ser interrompidas, resultando em atualizações incorretas ou valores inesperados.

In [ ]:
# exemplo
soma = 0

Threads.@threads for i in 1:10
    global soma += i  # Várias threads acessam/modificam a variável ao mesmo tempo
end

println(soma)

###  como evitar?
  1. Uso de variáveis atômicas

In [ ]:
using Base.Threads

soma = Atomic{Int}(0)

Threads.@threads for i in 1:10
    atomic_add!(soma, i)  # Atualização segura
end

println(soma[])  # Acessa o valor real da variável atômica

  2. Uso de *locks*

In [ ]:
using Base.Threads

my_lock = ReentrantLock()  # Cria a trava
soma = 0

Threads.@threads for i in 1:10
    @lock my_lock global soma += i
end

println(soma)

### ⋆ exemplos com vetores
1. Paralelizando uma soma em um vetor

In [ ]:
using Base.Threads

# Vetor a ser processado
vetor = collect(1:1_000_000)

# Resultado final
soma_total = Atomic{Int}(0)

# Paralelização com Threads
Threads.@threads for i in 1:length(vetor)
    atomic_add!(soma_total, vetor[i])  # Cada thread soma parte do vetor de forma segura
end

println("Soma total: ", soma_total[])

2. Multiplicação de elementos de um vetor 

In [ ]:
using Base.Threads

# Vetor inicial
vetor = collect(1:1_000_000)
resultado = Vector{Int}(undef, length(vetor))

Threads.@threads for i in 1:length(vetor)
    resultado[i] = vetor[i] * 2  # Cada thread processa um elemento
end

println("Primeiros 10 resultados: ", resultado[1:10])

3. Filtragem de elementos pares de um vetor

In [ ]:

# Vetor inicial
vetor = collect(1:10^6)

# Vetor para armazenar os pares
pares = Atomic{Vector{Int}}(Vector{Int}())

# Paraleliza a operação
Threads.@threads for i in 1:length(vetor)
    if vetor[i] % 2 == 0
        # Adiciona o valor de forma segura
        lock = ReentrantLock()
        @lock lock push!(pares[], vetor[i])
    end
end

println("Primeiros 10 pares: ", pares[][1:10])

4. Produtor e Consumidor

In [1]:
using Base.Threads

# Buffer compartilhado e configurações
buffer = Vector{Int}()
buffer_size = 10

# Trava para sincronização
lock = ReentrantLock()

# Condições para controle do buffer
not_empty = Condition()  # Buffer não está vazio
not_full = Condition()   # Buffer não está cheio

# Produtor
function producer()
    for i in 1:20
        lock(lock) do
            # Espera enquanto o buffer está cheio
            while length(buffer) >= buffer_size
                wait(not_full)
            end

            # Produz e adiciona ao buffer
            push!(buffer, i)
            println("Produzido: $i | Buffer: $buffer")

            # Notifica que o buffer não está vazio
            notify(not_empty)
        end
        sleep(0.1)  # Simula tempo de produção
    end
end

# Consumidor
function consumer()
    for _ in 1:20
        lock(lock) do
            # Espera enquanto o buffer está vazio
            while isempty(buffer)
                wait(not_empty)
            end

            # Consome do buffer
            item = popfirst!(buffer)
            println("Consumido: $item | Buffer: $buffer")

            # Notifica que o buffer não está cheio
            notify(not_full)
        end
        sleep(0.2)  # Simula tempo de consumo
    end
end

# Executa produtor e consumidor em threads diferentes
t1 = @spawn producer()
t2 = @spawn consumer()

# Aguarda ambas as threads terminarem
fetch(t1)
fetch(t2)

TaskFailedException: TaskFailedException

    nested task error: MethodError: objects of type ReentrantLock are not callable
    The object of type `ReentrantLock` exists, but no method is defined for this combination of argument types when trying to treat it as a callable object.
    Stacktrace:
     [1] producer()
       @ Main c:\Users\6030a\Downloads\estudos\julia\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X26sZmlsZQ==.jl:17
     [2] (::var"#15#16")()
       @ Main c:\Users\6030a\Downloads\estudos\julia\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X26sZmlsZQ==.jl:55